In [203]:
import nltk
import random
from nltk.treeprettyprinter import TreePrettyPrinter

from nltk import Tree

In [168]:
grammar = nltk.data.load("../data/atis-grammar-cnf.cfg")
#run once
nltk.download("large_grammars")
sentences = nltk.data.load("grammars/large_grammars/atis_sentences.txt", "auto")
# print(sentences)
test_sentences = nltk.parse.util.extract_test_sentences(sentences)
# print(test_sentences)

[nltk_data] Downloading package large_grammars to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package large_grammars is already up-to-date!


In [3]:
no_parse = [test_sentences[index][0] for index in range(len(test_sentences)) if test_sentences[index][1] == 0 and len(test_sentences[index][0]) <= 9]
print(len(no_parse))
yes_parse = [test_sentences[index][0] for index in range(len(test_sentences)) if test_sentences[index][1] != 0 and len(test_sentences[index][0]) <= 7]
print(len(yes_parse))

10
18


In [4]:
random.seed(42)
grammatical = random.sample(yes_parse,10)
ungrammatical = random.sample(no_parse,10)
print(grammatical[0])
print(ungrammatical[0])

['show', 'the', 'flights', '.']
['what', 'is', 'the', 'flying', 'time', 'from', '.']


In [169]:
sentence = test_sentences[0][0]
sentence

['i',
 'need',
 'a',
 'flight',
 'from',
 'charlotte',
 'to',
 'las',
 'vegas',
 'that',
 'makes',
 'a',
 'stop',
 'in',
 'saint',
 'louis',
 '.']

In [420]:
sentence_length = len(sentence)
print("Sentence Length :",sentence_length)
cky_matrix = [[set() for _ in range(sentence_length+1)] for _ in range(sentence_length+1)]
backpointers = [[[] for _ in range(sentence_length+1)] for _ in range(sentence_length+1)]

# print("Number of rows : ",len(cky_matrix))
# print("Number of columns : ",len(cky_matrix[:][0]))

# assert cky_matrix[0][sentence_length] == ''
    
for index, word in enumerate(sentence):
        cky_matrix[index][index] = word
        word_rules = grammar.productions(rhs = word)
        cky_matrix[index][index+1] = set([rule.lhs() for rule in word_rules])
        backpointers[index][index+1].append([(index,index,index+1,rule.lhs(),word,None) for rule in word_rules])
    
# cky_matrix
all_rules = grammar.productions()

for width in range(2,sentence_length+1):
    # print(width)
    for start in range(sentence_length - width + 1):
        end = start + width
        for mid in range(start + 1, end):
            for rule in all_rules:
                if len(rule.rhs()) == 2:
                    B, C = rule.rhs()
                    if B in cky_matrix[start][mid] and C in cky_matrix[mid][end]:
                        cky_matrix[start][end].add(rule.lhs())
                        backpointers[start][end].append((start,mid,end,rule.lhs(),B,C))
                        # print(backpointers[start][end][rule.lhs()])
                        # try:
                        #     backpointers[start][end].append((start,mid,end,B,C))
                            
                        # except KeyError:
                        #     backpointers[start][end][rule.lhs()] = []
                        #     backpointers[start][end][rule.lhs()].append((start,mid,end,B,C))
                            
start_symbol = grammar.start()

print(start_symbol in cky_matrix[0][sentence_length])

Sentence Length : 17
True


In [414]:
# def build_tree(start, end, symbol):
#     if start == end - 1:
#         return Tree(symbol, [sentence[start]])
#     else:
#         mid = backpointers[start][end][(symbol, (B, C))]
#         B, C = (B, C)
#         left_subtree = build_tree(start, mid, B)
#         right_subtree = build_tree(mid, end, C)
#         return Tree(symbol, [left_subtree, right_subtree])

In [421]:
cky_matrix[0][17]

{APB, CL, COMPCL_MD, DAS, DECL_MD, GGT, GHH, GHN, JIF, SIGMA, VP_MD}

In [430]:
backpointers[1][2]

[[(1, 1, 2, pt_verb_md, 'need', None), (1, 1, 2, VERB_MD, 'need', None)]]

In [433]:
[x for x in backpointers[0][1][0] if x[3] == nltk.grammar.Nonterminal("NP_PPSS")]

[(0, 0, 1, NP_PPSS, 'i', None)]

In [397]:
terminal_list = [item for item in backpointers[0][sentence_length] if item[3] == start_symbol]
terminal_list

[(0, 1, 17, SIGMA, NP_PPSS, AUE),
 (0, 1, 17, SIGMA, NP_PPSS, AUM),
 (0, 1, 17, SIGMA, NP_PPSS, DBN),
 (0, 1, 17, SIGMA, NP_PPSS, DBQ),
 (0, 1, 17, SIGMA, NP_PPSS, DBW),
 (0, 1, 17, SIGMA, NP_PPSS, DCC),
 (0, 1, 17, SIGMA, NP_PPSS, DCF),
 (0, 1, 17, SIGMA, NP_PPSS, DCR),
 (0, 1, 17, SIGMA, NP_PPSS, GJW),
 (0, 1, 17, SIGMA, NP_PPSS, GKA),
 (0, 1, 17, SIGMA, NP_PPSS, MU)]

In [444]:
end_list = set([(i,i+1) for i in range(sentence_length)])
word_list = set([(i,i) for i in range(sentence_length)])
all_trees = []

for element in terminal_list:
    flag = True
    roots = [element]
    checked_list = []
    
    while flag:
        for root in roots:
            print(root)
            if type(root) != str:
                if root[:2] in word_list:
                    print("oof")
                    checked_list.append(root[1:3])
                    # roots.append(root[4])
                    continue
                elif (root[0],root[2]) in checked_list:
                    print("oof2")
                    continue
                else:
                    print(f"getting children for {root}")
                    next_x, next_y = root[-2:]
                    checked_list.append((root[0],root[2]))
                    if root[:2] in end_list:
                        roots.append([x for x in backpointers[root[0]][root[1]][0] if x[3] == next_x][0])
                    else:
                        roots.append([x for x in backpointers[root[0]][root[1]] if x[3] == next_x][0])
                    if root[1:3] in end_list:
                        roots.append([y for y in backpointers[root[1]][root[2]][0] if y[3] == next_y][0])
                    else:
                        roots.append([y for y in backpointers[root[1]][root[2]] if y[3] == next_y][0])
                    
                    print(roots)
                    print(checked_list)
                    
            # assert type(roots) == list
                
            final_list = [(root[0],root[2]) for root in roots]
            print("Checking")
            flag = not(set(end_list) == set([value for value in final_list if value in end_list]))
            # prin
            
    all_trees.append(roots)    

# for element in terminal_list:
#     flag = True
#     roots = [element]
#     checked_list = []
#     # limit = 100
#     # i = 0
    
#     while flag:
#         # i+=1
#         # if i > limit:
#         #     break
#         for root in roots:
#             print(root)
#             if type(root) != str:
#                 if root[:2] in word_list:
#                     print("oof")
#                     checked_list.append(root[1:3])
#                     continue
#                 elif (root[0],root[2]) in checked_list:
#                     print("oof2")
#                     continue
#                 else:
#                     print(f"getting children for {root}")
#                     next_x, next_y = get_children(cky_matrix, root)
#                     checked_list.append((root[0],root[2]))
#                     roots.append(next_x)
#                     roots.append(next_y)
#                     print(roots)
#                     print(checked_list)
                
#         assert type(roots) == list
            
#         final_list = [(root[0],root[2]) for root in roots]
#         print("Checking")
#         flag = not(set(end_list) == set([value for value in final_list if value in end_list]))
#         print(flag)
        
#     all_trees.append(roots)    

(0, 1, 17, SIGMA, NP_PPSS, AUE)
getting children for (0, 1, 17, SIGMA, NP_PPSS, AUE)
[(0, 1, 17, SIGMA, NP_PPSS, AUE), (0, 0, 1, NP_PPSS, 'i', None), (1, 2, 17, AUE, VERB_MD, AUF)]
[(0, 17)]
Checking
(0, 0, 1, NP_PPSS, 'i', None)
oof
(1, 2, 17, AUE, VERB_MD, AUF)
getting children for (1, 2, 17, AUE, VERB_MD, AUF)
[(0, 1, 17, SIGMA, NP_PPSS, AUE), (0, 0, 1, NP_PPSS, 'i', None), (1, 2, 17, AUE, VERB_MD, AUF), (1, 1, 2, VERB_MD, 'need', None), (2, 4, 17, AUF, NP_NN, AUG)]
[(0, 17), (0, 1), (1, 17)]
Checking
(1, 1, 2, VERB_MD, 'need', None)
oof
(2, 4, 17, AUF, NP_NN, AUG)
getting children for (2, 4, 17, AUF, NP_NN, AUG)
[(0, 1, 17, SIGMA, NP_PPSS, AUE), (0, 0, 1, NP_PPSS, 'i', None), (1, 2, 17, AUE, VERB_MD, AUF), (1, 1, 2, VERB_MD, 'need', None), (2, 4, 17, AUF, NP_NN, AUG), (2, 3, 4, NP_NN, ADJ_AT, NOUN_NN), (4, 6, 17, AUG, PP_NP, AUH)]
[(0, 17), (0, 1), (1, 17), (1, 2), (2, 17)]
Checking
(2, 3, 4, NP_NN, ADJ_AT, NOUN_NN)
getting children for (2, 3, 4, NP_NN, ADJ_AT, NOUN_NN)
[(0, 1, 17,

In [446]:
len(all_trees)

11

In [447]:
imp_info = [[(leaf[0],leaf[2],leaf[3]) for leaf in tree] for tree in all_trees]
imp_info

[[(0, 17, SIGMA),
  (0, 1, NP_PPSS),
  (1, 17, AUE),
  (1, 2, VERB_MD),
  (2, 17, AUF),
  (2, 4, NP_NN),
  (4, 17, AUG),
  (2, 3, ADJ_AT),
  (3, 4, NOUN_NN),
  (4, 6, PP_NP),
  (6, 17, AUH),
  (4, 5, PREP_IN),
  (5, 6, NOUN_NP),
  (6, 16, PP_NN),
  (16, 17, pt_char_per),
  (6, 7, PREP_IN),
  (7, 16, IMO),
  (7, 12, NP_NP),
  (12, 16, IMP),
  (7, 9, NOUN_NP),
  (9, 12, RELCL_VBZ),
  (12, 13, NOUN_NN),
  (13, 16, PP_NP),
  (7, 8, las),
  (8, 9, vegas),
  (9, 10, NP_WPS),
  (10, 12, KRC),
  (13, 14, PREP_IN),
  (14, 16, KKX),
  (10, 11, VERB_VBZ),
  (11, 12, NP_NP),
  (14, 15, NOUN_NP),
  (15, 16, AVPNP_NP)],
 [(0, 17, SIGMA),
  (0, 1, NP_PPSS),
  (1, 17, AUM),
  (1, 2, VERB_MD),
  (2, 17, AUN),
  (2, 4, NP_NP),
  (4, 17, AUO),
  (2, 3, NOUN_NP),
  (3, 4, INFCL_VB),
  (4, 6, PP_NP),
  (6, 17, AUP),
  (4, 5, PREP_IN),
  (5, 6, NOUN_NP),
  (6, 16, PP_NN),
  (16, 17, pt_char_per),
  (6, 7, PREP_IN),
  (7, 16, IMO),
  (7, 12, NP_NP),
  (12, 16, IMP),
  (7, 9, NOUN_NP),
  (9, 12, RELCL_VBZ),
 

In [514]:
def form_branch(node,node_list):
    a = node[0]
    c = node[1]
    # output_child1, output_child2 = 0, 0 
    candidate_children = [check_node for check_node in node_list if (check_node[0] == a or check_node[1] == c) and check_node != node]
    # print(candidate_children)
    for child1 in candidate_children:
        # print(child1)
        # x = child[0]
        common = child1[1]
        other_candidates = [x for x in candidate_children if x != child1 and x != node]
        for child2 in other_candidates:
            # print(child2)
            if common == child2[0]:
                output_child1 = child1
                output_child2 = child2
                                
                return output_child1[-1], output_child2[-1], nltk.Tree(node = str(node[-1]) , children = [str(output_child1[-1]),str(output_child2[-1])])
    # return output_child1, output_child2
    
def form_tree(node_list):
    start_node = [node for node in node_list if node[:2] == (0,sentence_length)][0]
    # print(start_node)
    checked_list = []
    parent_child = []
    for node in node_list:
        if node not in checked_list:
            # print(parent_child)
            # print(checked_list)
            # print(node)
            if node[:2] in end_list:
                checked_list.append(node)
                parent_child.append([node[-1],[sentence[node[0]]]])
                # print("oof")
            else:
                child1, child2, tree_element = form_branch(node, node_list)
                checked_list.append(node)
                parent_child.append([node[-1],[child1,child2]])
            
    print(parent_child)
    branch_dict = {}
    for branch in parent_child:
        branch_dict[branch[0]] = branch[-1]
    print(branch_dict)
        
    return branch_dict    
        
def build_tree(grammar_dict, start_symbol):
    if start_symbol not in grammar_dict:
        return start_symbol  # Terminal symbol

    children = grammar_dict[start_symbol]
    if len(children) == 1 and isinstance(children[0], str):
        return Tree(start_symbol, [children[0]])

    subtrees = [build_tree(grammar_dict, child) for child in children]
    return Tree(start_symbol, subtrees)
    # # output_list = [nltk.Tree(node = branch_dict[str(start_node[-1])][0], children = branch_dict[str(start_node[-1])][1:])]
    # output_str = str([start_node[-1]])
    # checked_list = []
    # to_check = [str(start_node[-1])]

    # # while flag:    
    # # for node in parent_child:
    # #     if node[0] not in checked_list:
    # #         # print(checked_list)
    # #         checked_list.append(node[0])
    # #         nonterm1,nonterm2 = node[1]
    # #         # print(nonterm1,nonterm2)
    # #         if nonterm1 in branch_dict.keys():
    # #             # print("hey1")
    # #             # node[1][0] = nltk.Tree(node = branch_dict[nonterm1][0], children = branch_dict[nonterm1][1:])
    # #             val = branch_dict[nonterm1]
    # #             node[1][0] = *val
    # #         if nonterm2 in branch_dict.keys():
    # #             # print("hey2")
    # #             # node[1][1] = nltk.Tree(node = branch_dict[nonterm2][0], children = branch_dict[nonterm2][1:])
    # #             val = branch_dict[nonterm2]
    # #             node[1][1] = *val
    # # for head in branch_dict.keys():
    # #     if head in output_list[1]:
    # flag = True
    # while flag:
    #     for head in branch_dict.keys():
    #         if head in to_check:
    #             head_index = output_str.rfind(head)
    #             print(head, head_index)
    #             if head_index != -1:
    #                 output_str = output_str[:head_index] + head + " " + str(branch_dict[head]) + output_str[head_index + len(head):]
    #             # output_str = output_str.replace(head , head + " " + branch_dict[head])
    #                 to_check.remove(head)
    #                 if isinstance(branch_dict[head][0],str):
    #                     continue 
    #                 else:
    #                     to_check.append(str(branch_dict[head][0]))
    #                 try:
    #                     to_check.append(str(branch_dict[head][1]))
    #                 except IndexError:
    #                     continue
    #             else:
    #                 continue
    #         print(to_check)
    #         # print(branch_dict.keys())
    #         print(output_str)
    #         if set(checked_list) == set(branch_dict.keys()):
    #             flag = False
            
                
    # return output_str

In [515]:
branch_dict = form_tree(imp_info[0])

# tree = nltk.Tree.fromstring(tree,brackets='[]')

[[SIGMA, [NP_PPSS, AUE]], [NP_PPSS, ['i']], [AUE, [VERB_MD, AUF]], [VERB_MD, ['need']], [AUF, [NP_NN, AUG]], [NP_NN, [ADJ_AT, NOUN_NN]], [AUG, [PP_NP, AUH]], [ADJ_AT, ['a']], [NOUN_NN, ['flight']], [PP_NP, [PREP_IN, NOUN_NP]], [AUH, [PP_NN, pt_char_per]], [PREP_IN, ['from']], [NOUN_NP, ['charlotte']], [PP_NN, [PREP_IN, IMO]], [pt_char_per, ['.']], [PREP_IN, ['to']], [IMO, [NP_NP, IMP]], [NP_NP, [NOUN_NP, RELCL_VBZ]], [IMP, [NOUN_NN, PP_NP]], [NOUN_NP, [las, vegas]], [RELCL_VBZ, [NP_WPS, KRC]], [NOUN_NN, ['stop']], [PP_NP, [PREP_IN, KKX]], [las, ['las']], [vegas, ['vegas']], [NP_WPS, ['that']], [KRC, [VERB_VBZ, NP_NP]], [PREP_IN, ['in']], [KKX, [NOUN_NP, AVPNP_NP]], [VERB_VBZ, ['makes']], [NP_NP, ['a']], [NOUN_NP, ['saint']], [AVPNP_NP, ['louis']]]
{SIGMA: [NP_PPSS, AUE], NP_PPSS: ['i'], AUE: [VERB_MD, AUF], VERB_MD: ['need'], AUF: [NP_NN, AUG], NP_NN: [ADJ_AT, NOUN_NN], AUG: [PP_NP, AUH], ADJ_AT: ['a'], NOUN_NN: ['stop'], PP_NP: [PREP_IN, KKX], AUH: [PP_NN, pt_char_per], PREP_IN: ['in'

In [516]:
start_symbol

SIGMA

In [518]:
tree = build_tree(branch_dict,start_symbol)

In [521]:
tree

The Ghostscript executable isn't found.
See http://web.mit.edu/ghostscript/www/Install.htm
If you're using a Mac, you can try installing
https://docs.brew.sh/Installation then `brew install ghostscript`


LookupError: 

Tree(SIGMA, [Tree(NP_PPSS, ['i']), Tree(AUE, [Tree(VERB_MD, ['need']), Tree(AUF, [Tree(NP_NN, [Tree(ADJ_AT, ['a']), Tree(NOUN_NN, ['stop'])]), Tree(AUG, [Tree(PP_NP, [Tree(PREP_IN, ['in']), Tree(KKX, [Tree(NOUN_NP, ['saint']), Tree(AVPNP_NP, ['louis'])])]), Tree(AUH, [Tree(PP_NN, [Tree(PREP_IN, ['in']), Tree(IMO, [Tree(NP_NP, ['a']), Tree(IMP, [Tree(NOUN_NN, ['stop']), Tree(PP_NP, [Tree(PREP_IN, ['in']), Tree(KKX, [Tree(NOUN_NP, ['saint']), Tree(AVPNP_NP, ['louis'])])])])])]), Tree(pt_char_per, ['.'])])])])])])

In [519]:
print(TreePrettyPrinter(tree).text())

TypeError: object of type 'Nonterminal' has no len()

In [ ]:
out = nltk.ImmutableTree.convert(tree=tree)

In [432]:
terminal_list[0][1:3]

(1, 17)

In [348]:
def get_children(backpointers, element):
    
    a, b, c = element[0],element[1],element[2],element[-2],element[-1]

    next_x = matrix[a][b][indx]
    next_y = matrix[b][c][indy]
    
    return next_x, next_y


def build_tree(start, end, symbol):
    checked_list = []
    out_trees = []
    out_string = ""
    if start == end - 1:
        if symbol not in checked_list:
            checked_list.append(symbol)
            out_string += f"{str(symbol)} ({sentence[start]})"
            # print(end)
            # return end
    else:
        # out_trees = []
        # for i in range(len(backpointers[start][end][symbol])):
            # print(i)
        mid, B, C = backpointers[start][end][symbol][0]
        # out_string += f"({symbol} "
        print(mid,B,C)
        if B not in checked_list:
            left_subtree = build_tree(start, mid, B)
            checked_list.append(B)
        if C not in checked_list:
            right_subtree = build_tree(mid, end, C)
            checked_list.append(C)
        
        
        out_string += f"({symbol} ({left_subtree} {right_subtree}))"
            
    return out_string

In [349]:
build_tree(0,1,nltk.grammar.Nonterminal("NP_PPSS"))

'NP_PPSS (i)'

In [350]:
parse_tree = build_tree(0,sentence_length,start_symbol)
# build_tree(0,sentence_length,start_symbol)

1 NP_PPSS AUE
2 VERB_MD AUF
4 NP_NN AUG
3 ADJ_AT NOUN_NN
ADJ_AT (a)
NOUN_NN (flight)
6 PP_NP AUH
5 PREP_IN NOUN_NP
PREP_IN (from)
NOUN_NP (charlotte)
16 PP_NN pt_char_per
7 PREP_IN IMO
12 NP_NP IMP
9 NOUN_NP RELCL_VBZ
8 las vegas
las (las)
vegas (vegas)
10 NP_WPS KRC
11 VERB_VBZ NP_NP
VERB_VBZ (makes)
NP_NP (a)
NP_WPS (that)
(KRC (VERB_VBZ (makes) NP_NP (a)))
(NOUN_NP (las (las) vegas (vegas)))
(RELCL_VBZ (NP_WPS (that) (KRC (VERB_VBZ (makes) NP_NP (a)))))
13 NOUN_NN PP_NP
14 PREP_IN KKX
15 NOUN_NP AVPNP_NP
NOUN_NP (saint)
AVPNP_NP (louis)
PREP_IN (in)
(KKX (NOUN_NP (saint) AVPNP_NP (louis)))
NOUN_NN (stop)
(PP_NP (PREP_IN (in) (KKX (NOUN_NP (saint) AVPNP_NP (louis)))))
(NP_NP ((NOUN_NP (las (las) vegas (vegas))) (RELCL_VBZ (NP_WPS (that) (KRC (VERB_VBZ (makes) NP_NP (a)))))))
(IMP (NOUN_NN (stop) (PP_NP (PREP_IN (in) (KKX (NOUN_NP (saint) AVPNP_NP (louis)))))))
PREP_IN (to)
(IMO ((NP_NP ((NOUN_NP (las (las) vegas (vegas))) (RELCL_VBZ (NP_WPS (that) (KRC (VERB_VBZ (makes) NP_NP (a)))))

In [351]:
# parse_tree
print(TreePrettyPrinter(Tree.fromstring(parse_tree)).text())

                                SIGMA                                                                                                                                        
                                  |                                                                                                                                           
                               NP_PPSS                                                                                                                                       
  ________________________________|_______________________________________________                                                                                            
 |                                                                               AUE                                                                                         
 |                                                                                |                                             

In [290]:
# print(TreePrettyPrinter(parse_tree[0]).text())
t = parse_tree[0].replace("[","").replace("]","").replace("\\","").replace("\'","").replace('"',"")
print(TreePrettyPrinter(Tree.fromstring(t)).text())

                                                                                                                                                                                                                               SIGMA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
                      

In [188]:
nltk.grammar.Nonterminal("SIGMA") in list(cky_matrix[0][1])[:][3]

False

In [45]:
sentence = grammatical[0]
" ".join(sentence)

'show the flights .'

In [46]:
sentence_length = len(sentence)
print("Sentence Length :",sentence_length)
cky_matrix = [["" for x in range(sentence_length+1)] for x in range(sentence_length)]

print("Number of rows : ",len(cky_matrix))
print("Number of columns : ",len(cky_matrix[:][0]))

assert cky_matrix[0][sentence_length] == ''

Sentence Length : 4
Number of rows :  4
Number of columns :  5


In [47]:
for index, word in enumerate(sentence):
    cky_matrix[index][index] = word
    word_rules = grammar.productions(rhs = word)
    cky_matrix[index][index+1] = [(index,index,index+1,rule.lhs(),index,index) for rule in word_rules]

In [48]:
# cky_matrix
all_rules = grammar.productions()

for b in range(2,sentence_length+2):
    for i in range(1,sentence_length-b+1):
        cky_matrix[i][i+b] = []
        for k in range(1,b):
            # print(b,i,k)
            B = cky_matrix[i][i+k]
            C = cky_matrix[i+k][i+b]
            # print(B)
            # print(C)
            # all_lhs = []
            
            for indx, x in enumerate(B):
                for indy, y in enumerate(C):
                    # print(x,y)
                    lefts = [(i,i+k,i+b,rules.lhs(),indx,indy) for rules in all_rules if rules.rhs() == (x[3],y[3])]
                    lefts = list(set(lefts))
                    # print(lefts)
                    # print(lefts)
                    for left in lefts:
                        if left not in cky_matrix[i][i+b]:
                            cky_matrix[i][i+b].append(left)
                            
for b in range(2,sentence_length+1):
    i = 0
    cky_matrix[i][i+b] = []
    for k in range(1,b):
        # print(b,i,k)
        B = cky_matrix[i][i+k]
        C = cky_matrix[i+k][i+b]
        # print(B)
        # print(C)
        # all_lhs = []
        
        for indx, x in enumerate(B):
            for indy, y in enumerate(C):
                # print(x,y)
                lefts = [(i,i+k,i+b,rules.lhs(),indx,indy) for rules in all_rules if rules.rhs() == (x[3],y[3])]
                lefts = list(set(lefts))
                # print(lefts)
                # print(lefts)
                for left in lefts:
                    if left not in cky_matrix[i][i+b]:
                        cky_matrix[i][i+b].append(left)
                        
# if nltk.grammar.Nonterminal("SIGMA") in cky_matrix[0][sentence_length]:
    # print(f"True, there is the Start symbol in index {0},{sentence_length}")
    # return True
# else:
    # print(f"False, there is no Start symbol in index {0},{sentence_length}")
    # return False

In [49]:
cky_matrix

[['show',
  [(0, 0, 1, VP_VB, 0, 0),
   (0, 0, 1, NOUN_NN, 0, 0),
   (0, 0, 1, INFCL_VB, 0, 0),
   (0, 0, 1, NP_NN, 0, 0),
   (0, 0, 1, SIGMA, 0, 0),
   (0, 0, 1, AVPNP_NN, 0, 0),
   (0, 0, 1, show, 0, 0),
   (0, 0, 1, VERB_VB, 0, 0)],
  [(0, 1, 2, JWU, 1, 2),
   (0, 1, 2, JVQ, 1, 2),
   (0, 1, 2, KIJ, 1, 2),
   (0, 1, 2, FSV, 3, 2),
   (0, 1, 2, GZR, 3, 2),
   (0, 1, 2, FZT, 3, 2),
   (0, 1, 2, GZJ, 3, 2),
   (0, 1, 2, JKU, 7, 2),
   (0, 1, 2, IZF, 7, 2),
   (0, 1, 2, VP_VB, 7, 2)],
  [(0, 1, 3, FVD, 7, 4),
   (0, 1, 3, KGI, 7, 4),
   (0, 1, 3, JLB, 7, 4),
   (0, 1, 3, JAX, 7, 4),
   (0, 1, 3, JHQ, 7, 4),
   (0, 1, 3, KQH, 7, 4),
   (0, 1, 3, VP_VB, 7, 4),
   (0, 1, 3, KSI, 7, 4),
   (0, 1, 3, KSI, 7, 10),
   (0, 1, 3, FVD, 7, 10),
   (0, 1, 3, KGI, 7, 10),
   (0, 1, 3, JLB, 7, 10),
   (0, 1, 3, JAX, 7, 10),
   (0, 1, 3, KQH, 7, 10),
   (0, 1, 3, VP_VB, 7, 10),
   (0, 1, 3, JHQ, 7, 10)],
  [(0, 1, 4, KGI, 7, 0),
   (0, 1, 4, FVD, 7, 0),
   (0, 1, 4, JLB, 7, 0),
   (0, 1, 4, JHQ, 7, 0)

In [50]:
terminal_list = [item for item in cky_matrix[0][sentence_length] if item[3] == nltk.grammar.Nonterminal("SIGMA")]
terminal_list

[(0, 1, 4, SIGMA, 7, 39), (0, 1, 4, SIGMA, 7, 129)]

In [51]:
# print(cky_matrix[1][2][1])
# print(cky_matrix[2][3][3])
# print(cky_matrix[1][2][1])
# print(cky_matrix[2][3][3])

In [52]:
def get_children(matrix, element):
    a,b,c,indx,indy = element[0],element[1],element[2],element[-2],element[-1]

    next_x = matrix[a][b][indx]
    next_y = matrix[b][c][indy]
    
    return next_x, next_y

In [53]:
end_list = set([(i,i+1) for i in range(sentence_length)])
word_list = set([(i,i) for i in range(sentence_length)])
all_trees = []

for element in terminal_list:
    flag = True
    roots = [element]
    checked_list = []
    # limit = 100
    # i = 0
    
    while flag:
        # i+=1
        # if i > limit:
        #     break
        for root in roots:
            print(root)
            if type(root) != str:
                if root[:2] in word_list:
                    print("oof")
                    checked_list.append(root[1:3])
                    continue
                elif (root[0],root[2]) in checked_list:
                    print("oof2")
                    continue
                else:
                    print(f"getting children for {root}")
                    next_x, next_y = get_children(cky_matrix, root)
                    checked_list.append((root[0],root[2]))
                    roots.append(next_x)
                    roots.append(next_y)
                    print(roots)
                    print(checked_list)
                
        assert type(roots) == list
            
        final_list = [(root[0],root[2]) for root in roots]
        print("Checking")
        flag = not(set(end_list) == set([value for value in final_list if value in end_list]))
        print(flag)
        
    all_trees.append(roots)

(0, 1, 4, SIGMA, 7, 39)
getting children for (0, 1, 4, SIGMA, 7, 39)
[(0, 1, 4, SIGMA, 7, 39), (0, 0, 1, VERB_VB, 0, 0), (1, 3, 4, JKB, 4, 0)]
[(0, 4)]
(0, 0, 1, VERB_VB, 0, 0)
oof
(1, 3, 4, JKB, 4, 0)
getting children for (1, 3, 4, JKB, 4, 0)
[(0, 1, 4, SIGMA, 7, 39), (0, 0, 1, VERB_VB, 0, 0), (1, 3, 4, JKB, 4, 0), (1, 2, 3, NP_NNS, 1, 3), (3, 3, 4, pt_char_per, 3, 3)]
[(0, 4), (0, 1), (1, 4)]
(1, 2, 3, NP_NNS, 1, 3)
getting children for (1, 2, 3, NP_NNS, 1, 3)
[(0, 1, 4, SIGMA, 7, 39), (0, 0, 1, VERB_VB, 0, 0), (1, 3, 4, JKB, 4, 0), (1, 2, 3, NP_NNS, 1, 3), (3, 3, 4, pt_char_per, 3, 3), (1, 1, 2, ADJ_AT, 1, 1), (2, 2, 3, NOUN_NNS, 2, 2)]
[(0, 4), (0, 1), (1, 4), (1, 3)]
(3, 3, 4, pt_char_per, 3, 3)
oof
(1, 1, 2, ADJ_AT, 1, 1)
oof
(2, 2, 3, NOUN_NNS, 2, 2)
oof
Checking
False
(0, 1, 4, SIGMA, 7, 129)
getting children for (0, 1, 4, SIGMA, 7, 129)
[(0, 1, 4, SIGMA, 7, 129), (0, 0, 1, VERB_VB, 0, 0), (1, 3, 4, JKB, 10, 0)]
[(0, 4)]
(0, 0, 1, VERB_VB, 0, 0)
oof
(1, 3, 4, JKB, 10, 0)
gettin

In [54]:
all_trees

[[(0, 1, 4, SIGMA, 7, 39),
  (0, 0, 1, VERB_VB, 0, 0),
  (1, 3, 4, JKB, 4, 0),
  (1, 2, 3, NP_NNS, 1, 3),
  (3, 3, 4, pt_char_per, 3, 3),
  (1, 1, 2, ADJ_AT, 1, 1),
  (2, 2, 3, NOUN_NNS, 2, 2)],
 [(0, 1, 4, SIGMA, 7, 129),
  (0, 0, 1, VERB_VB, 0, 0),
  (1, 3, 4, JKB, 10, 0),
  (1, 2, 3, NP_NNS, 2, 3),
  (3, 3, 4, pt_char_per, 3, 3),
  (1, 1, 2, AVP_RB, 1, 1),
  (2, 2, 3, NOUN_NNS, 2, 2)]]

In [55]:
# assert type(roots) == set

In [56]:
# set(end_list) == set([value for value in final_list if value in end_list])

In [57]:
# cky_matrix[1][2][0][1:3]

In [58]:
# roots

In [59]:
# print(cky_matrix[0][1][7])
# print(cky_matrix[1][4][64])
# print(cky_matrix[1][3][7])
# print(cky_matrix[3][4][0])

In [60]:
# test_list = [(0,0),(0,9),(1,1),(2,3),(2,2),(1,3)]
# end_list = [(0,0),(1,1),(2,2),(3,3)]
# val = not(end_list == [value for value in test_list if value in end_list])
# val

In [61]:
# True | False

In [90]:
imp_info = [[(leaf[0],leaf[2],leaf[3]) for leaf in tree] for tree in all_trees]

In [91]:
imp_info

[[(0, 4, SIGMA),
  (0, 1, VERB_VB),
  (1, 4, JKB),
  (1, 3, NP_NNS),
  (3, 4, pt_char_per),
  (1, 2, ADJ_AT),
  (2, 3, NOUN_NNS)],
 [(0, 4, SIGMA),
  (0, 1, VERB_VB),
  (1, 4, JKB),
  (1, 3, NP_NNS),
  (3, 4, pt_char_per),
  (1, 2, AVP_RB),
  (2, 3, NOUN_NNS)]]

In [153]:
def form_branch(node,node_list):
    a = node[0]
    c = node[1]
    # output_child1, output_child2 = 0, 0 
    candidate_children = [check_node for check_node in node_list if (check_node[0] == a or check_node[1] == c) and check_node != node]
    # print(candidate_children)
    for child1 in candidate_children:
        # print(child1)
        # x = child[0]
        common = child1[1]
        other_candidates = [x for x in candidate_children if x != child1 and x != node]
        for child2 in other_candidates:
            # print(child2)
            if common == child2[0]:
                output_child1 = child1
                output_child2 = child2
                                
                return output_child1[-1], output_child2[-1], nltk.Tree(node = str(node[-1]) , children = [str(output_child1[-1]),str(output_child2[-1])])
    # return output_child1, output_child2
    
def form_tree(node_list):
    start_node = [node for node in node_list if node[:2] == (0,sentence_length)][0]
    print(start_node)
    checked_list = []
    parent_child = []
    for node in node_list:
        if node not in checked_list:
            # print(parent_child)
            # print(checked_list)
            # print(node)
            if node[:2] in end_list:
                checked_list.append(node)
                # print("oof")
            else:
                child1, child2, tree_element = form_branch(node, node_list)
                checked_list.append(node)
                parent_child.append([node[-1],[child1,child2]])
            
    print(parent_child)
    branch_dict = {}
    for branch in parent_child:
        branch_dict[str(branch[0])] = str(branch[-1])
    print(branch_dict)
        
    # output_list = [nltk.Tree(node = branch_dict[str(start_node[-1])][0], children = branch_dict[str(start_node[-1])][1:])]
    output_str = str([start_node[-1]])
    checked_list = []
    
    # while flag:    
    # for node in parent_child:
    #     if node[0] not in checked_list:
    #         # print(checked_list)
    #         checked_list.append(node[0])
    #         nonterm1,nonterm2 = node[1]
    #         # print(nonterm1,nonterm2)
    #         if nonterm1 in branch_dict.keys():
    #             # print("hey1")
    #             # node[1][0] = nltk.Tree(node = branch_dict[nonterm1][0], children = branch_dict[nonterm1][1:])
    #             val = branch_dict[nonterm1]
    #             node[1][0] = *val
    #         if nonterm2 in branch_dict.keys():
    #             # print("hey2")
    #             # node[1][1] = nltk.Tree(node = branch_dict[nonterm2][0], children = branch_dict[nonterm2][1:])
    #             val = branch_dict[nonterm2]
    #             node[1][1] = *val
    # for head in branch_dict.keys():
    #     if head in output_list[1]:
    flag = True
    while flag:
        for head in branch_dict.keys():
            if head in output_str and head not in checked_list:
                output_str = output_str.replace(head , head + " "+ branch_dict[head].replace(","," "))
                checked_list.append(head)
            if set(checked_list) == set(branch_dict.keys()):
                flag = False
            
                
    return output_str

In [154]:
imp_info[1]

[(0, 4, SIGMA),
 (0, 1, VERB_VB),
 (1, 4, JKB),
 (1, 3, NP_NNS),
 (3, 4, pt_char_per),
 (1, 2, AVP_RB),
 (2, 3, NOUN_NNS)]

In [158]:
tree = form_tree(imp_info[0])
tree = nltk.Tree.fromstring(tree,brackets='[]')

(0, 4, SIGMA)
[[SIGMA, [VERB_VB, JKB]], [JKB, [NP_NNS, pt_char_per]], [NP_NNS, [ADJ_AT, NOUN_NNS]]]
{'SIGMA': '[VERB_VB, JKB]', 'JKB': '[NP_NNS, pt_char_per]', 'NP_NNS': '[ADJ_AT, NOUN_NNS]'}


In [78]:
# print(nltk.Tree.fromstring(str(tree).replace(","," "),brackets= '[]'))

(SIGMA (VERB_VB (JKB ( (NP_NNS (ADJ_AT NOUN_NNS)) pt_char_per))))


In [156]:
nltk.Tree.fromstring(tree, brackets= '[]')

The Ghostscript executable isn't found.
See http://web.mit.edu/ghostscript/www/Install.htm
If you're using a Mac, you can try installing
https://docs.brew.sh/Installation then `brew install ghostscript`


LookupError: 

Tree('SIGMA', [Tree('VERB_VB', ['JKB', Tree('NP_NNS', [Tree('ADJ_AT', ['NOUN_NNS']), 'pt_char_per'])])])

In [157]:
print(TreePrettyPrinter(nltk.Tree.fromstring(tree,brackets='[]')).text())

       SIGMA                   
         |                      
      VERB_VB                  
  _______|________              
 |              NP_NNS         
 |        ________|_______      
 |       |              ADJ_AT 
 |       |                |     
JKB pt_char_per        NOUN_NNS



In [ ]:
branch_dict = {}
for branch in tree:
    branch_dict[branch[0]] = branch

In [ ]:
"JKB" in branch_dict.keys()

True

In [ ]:
t = ['SIGMA', 'JKB', 'NP_NNS']

In [ ]:
t = nltk.Tree(node = branch_dict["SIGMA"][0], children = branch_dict["SIGMA"][1:])

In [ ]:
t

The Ghostscript executable isn't found.
See http://web.mit.edu/ghostscript/www/Install.htm
If you're using a Mac, you can try installing
https://docs.brew.sh/Installation then `brew install ghostscript`


LookupError: 

Tree('SIGMA', [('VERB_VB', 'JKB')])

In [160]:
type(tree) == nltk.ImmutableTree

False

In [161]:
print(TreePrettyPrinter(tree).text())

       SIGMA                   
         |                      
      VERB_VB                  
  _______|________              
 |              NP_NNS         
 |        ________|_______      
 |       |              ADJ_AT 
 |       |                |     
JKB pt_char_per        NOUN_NNS



In [ ]:
# t.ImmutableTree(node = )

In [162]:
nltk.ImmutableTree(tree.node)

TypeError: ImmutableTree: Expected a node value and child list 

In [164]:
type(tree)

nltk.tree.Tree

In [166]:
out = nltk.ImmutableTree.convert(tree=tree)

In [167]:
type(out) == nltk.ImmutableTree

True